<a href="https://colab.research.google.com/github/renardelyon/Feed-Formulation/blob/main/Feed%20Optimization.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install -U pymoo

In [2]:
import numpy as np
from pymoo.model.problem import Problem
from pymoo.algorithms.nsga2 import NSGA2
from pymoo.factory import get_sampling, get_crossover, get_mutation
from pymoo.factory import get_termination
from pymoo.optimize import minimize

### MyProblem Class

In [3]:
class MyProblem(Problem):
    def __init__(self, w_air1, w_air2, w_air3, w_air4,
                 w_pro1, w_pro2, w_pro3, w_pro4,
                 w_lemak1, w_lemak2, w_lemak3, w_lemak4,
                 w_sk1, w_sk2, w_sk3, w_sk4,
                 P1, P2, P3, P4):
        super().__init__(n_var=4,
                         n_obj=5,
                         n_constr=1,
                         xl=np.array([100, 100, 100, 100]),
                         xu=np.array([1000, 1000, 1000, 1000]))
        
        self.w_air1 = w_air1
        self.w_air2 = w_air2
        self.w_air3 = w_air3
        self.w_air4 = w_air4
        self.w_pro1 = w_pro1
        self.w_pro2 = w_pro2
        self.w_pro3 = w_pro3
        self.w_pro4 = w_pro4
        self.w_lemak1 = w_lemak1
        self.w_lemak2 = w_lemak2
        self.w_lemak3 = w_lemak3
        self.w_lemak4 = w_lemak4
        self.w_sk1 = w_sk1
        self.w_sk2 = w_sk1
        self.w_sk3 = w_sk1
        self.w_sk4 = w_sk1
        self.P1 = P1
        self.P2 = P2
        self.P3 = P3
        self.P4 = P4

    def _evaluate(self, X, out, *args, **kwargs):
        f1 = -self.w_pro1*X[:,0] - self.w_pro2*X[:,1] - self.w_pro3*X[:,2] \
              - self.w_pro4*X[:,3]
        f2 = self.w_lemak1*X[:,0] + self.w_lemak2*X[:,1] + self.w_lemak3*X[:,2]\
             + self.w_lemak4*X[:,3]
        f3 = self.w_air1*X[:,0] + self.w_air2*X[:,1] + self.w_air3*X[:,2]\
             + self.w_air4*X[:,3]
        f4 = self.w_sk1*X[:,0] + self.w_sk2*X[:,1] + self.w_sk3*X[:,2]\
             + self.w_sk4*X[:,3]
        f5 = self.P1*X[:,0] + self.P2*X[:,1] + self.P3*X[:,2]\
             + self.P4*X[:,3]

        g1 = (X[:,0] + X[:,1] + X[:,2] + X[:,3] - 1000)**2

        out["F"] = np.column_stack([f1, f2, f3, f4, f5])
        out["G"] = np.column_stack([g1])


###GAOptimization Class

In [7]:
class GA_Optimization:
  def __init__(self, pop_size, n_termination, seed):
    self.pop_size = pop_size
    self.n_termination = n_termination
    self.seed = seed 
  
  def get_algorithm(self):
    algorithm = NSGA2(pop_size=self.pop_size,
                      n_offsprings=10,
                      sampling=get_sampling("real_random"),
                      crossover=get_crossover("real_sbx", prob=0.9, eta=15),
                      mutation=get_mutation("real_pm", eta=20),
                      eliminate_duplicates=True)
    return algorithm 

  def get_termination(self):
    termination = get_termination("n_gen", self.n_termination)
    return termination

  def get_optimization(self, problem, algorithm, termination):
    res =  minimize(vectorized_problem,
               algorithm,
               termination,
               seed=self.seed,
               save_history=True,
               verbose=True)
    return res
    

### FeedInfo Class

In [64]:
class FeedInfo:
  def __init__(self, X):
    self.X = X
  
  def feed_ratio(self):
    ratio = self.X/1000.0
    return ratio
  
  def feed_composition(self):
    X = self.X
    f1 = w_pro1*X[0]  + w_pro2*X[1] + w_pro3*X[2] + w_pro4*X[3]
    f2 = w_lemak1*X[0] + w_lemak2*X[1] + w_lemak3*X[2] + w_lemak4*X[3]
    f3 = w_air1*X[0] + w_air2*X[1] + w_air3*X[2] + w_air4*X[3]
    f4 = w_sk1*X[0] + w_sk2*X[1] + w_sk3*X[2] + w_sk4*X[3]
    f5 = P1*X[0] + P2*X[1] + P3*X[2] + P4*X[3]
  
    composition = np.array([f1, f2, f3, f4,f5], dtype='float32')/10
    price = f5/1000
    return composition, price



### Bahan Baku yang digunakan

1.   Bekatul
1.   Ampas tahu
2.   Ampas kelapa 
2.   Tepung jagung



In [80]:
#@title Kadar air

w_air1=0.5 #@param
w_air2=0.7 #@param
w_air3=0.2 #@param
w_air4=0 #@param

In [81]:
#@title Kadar protein
w_pro1=0.12 #@param
w_pro2=0.077 #@param
w_pro3=0.08 #@param
w_pro4=0 #@param

In [82]:
#@title Kadar lemak
w_lemak1=0.107 #@param
w_lemak2=0.016 #@param
w_lemak3=0.29 #@param
w_lemak4=0 #@param

In [83]:
#@title Kadar serat kasar
w_sk1=0.052 #@param
w_sk2=0.064 #@param
w_sk3=0.13 #@param
w_sk4=0 #@param

In [84]:
#@title Harga bahan baku per kg
P1=3200 #@param
P2=1000 #@param
P3=300 #@param
P4=0 #@param

### Prediction

In [ ]:
vectorized_problem = MyProblem(w_air1, w_air2, w_air3, w_air4,
                 w_pro1, w_pro2, w_pro3, w_pro4,
                 w_lemak1, w_lemak2, w_lemak3, w_lemak4,
                 w_sk1, w_sk2, w_sk3, w_sk4,
                 P1, P2, P3, P4)
ga_optimization = GA_Optimization(100, 5000, np.random.randint(100))
algorithm = ga_optimization.get_algorithm()
termination = ga_optimization.get_termination()
res = ga_optimization.get_optimization(vectorized_problem,
                                       algorithm,
                                       termination)

In [86]:
X = res.X[0]
feed = FeedInfo(X)
ratio = feed.feed_ratio()
compostion, price = feed.feed_composition()

In [89]:
X

array([537.00404681, 220.08080476, 116.63753899, 126.27760944])

### Info Pakan

In [87]:
#@title Rasio  bahan baku yang dibutuhkan dan prediksi harga bahan baku per kg pakan yang dibuat
print('Kadar 1: {:.2f}\nKadar 2: {:.2f}'.format(ratio[0],ratio[1]))
print('Kadar 3: {:.2f}\nKadar 4: {:.2f}\n'.format(ratio[2],ratio[3]))
print('Harga bahan baku/kg pakan :Rp.{:.2f}'.format(price))

Kadar 1: 0.54
Kadar 2: 0.22
Kadar 3: 0.12
Kadar 4: 0.13

Harga bahan baku/kg pakan :Rp.1973.49


In [88]:
#@title Prediksi komposisi pakan
print('Kadar protein: {:.2f}%\nKadar lemak: {:.2f}%\nKadar air: {:.2f}%'.format(compostion[0],
                                                                          compostion[1],
                                                                          compostion[2]))
print('Kadar serat kasar: {:.2f}%'.format(compostion[3]))

Kadar protein: 9.07%
Kadar lemak: 9.48%
Kadar air: 44.59%
Kadar serat kasar: 5.72%
